In [2]:
import numpy as np
import tensorflow as tf

C:\Users\lewis\AppData\Local\Programs\Python\Python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)

test_data = mnist.test.images  # Returns np.array
test_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [5]:
train_labels

array([7, 3, 4, ..., 5, 6, 8])

## obejct detail [tf.reshape](https://www.tensorflow.org/api_docs/python/tf/reshape)

In [6]:
# tensor 't' is [[[1, 1, 1],
#                 [2, 2, 2]],
#                [[3, 3, 3],
#                 [4, 4, 4]],
#                [[5, 5, 5],
#                 [6, 6, 6]]]
# tensor 't' has shape [3, 2, 3]
# pass '[-1]' to flatten 't'
# tf.reshape(t, [-1]) ==> [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6]
# reshape(t, [2, -1]) ==> [[1, 1, 1, 2, 2, 2, 3, 3, 3],
#                          [4, 4, 4, 5, 5, 5, 6, 6, 6]]
# -1 代表著撐開所有數值

x = tf.reshape(train_data, [-1, 28, 28, 1])
y = train_labels

Convolutional Layer #1

Computes 32 features using a 5x5 filter with [ReLU activation](https://zh.wikipedia.org/wiki/线性整流函数).

Padding is added to preserve width and height.

Input Tensor Shape: [batch_size, 28, 28, 1]

Output Tensor Shape: [batch_size, 28, 28, 32]

In [7]:
# 把它捲及一個單位向量包含後五個單位向量來增加特徵

conv1 = tf.layers.conv2d(
    inputs=x,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

Pooling Layer #1

First max pooling layer with a 2x2 filter and stride of 2

Input Tensor Shape: [batch_size, 28, 28, 32]

Output Tensor Shape: [batch_size, 14, 14, 32]

## object detail : [tf.layers.max_pooling2d](https://www.tensorflow.org/api_docs/python/tf/layers/max_pooling2d)

In [8]:
# 再將兩行兩列的特徵合併成單一特徵

pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

捲及 (Convolutional) 增加單一資訊的特徵後，在使用池來組 (Pooling) 成特定特徵

下面在鏡行一次，來提取更高階級的特徵。

In [9]:
conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

In [10]:
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

In [11]:
conv1

<tf.Tensor 'conv2d/Relu:0' shape=(55000, 28, 28, 32) dtype=float32>

In [12]:
pool1

<tf.Tensor 'max_pooling2d/MaxPool:0' shape=(55000, 14, 14, 32) dtype=float32>

In [13]:
conv2

<tf.Tensor 'conv2d_1/Relu:0' shape=(55000, 14, 14, 64) dtype=float32>

In [14]:
pool2

<tf.Tensor 'max_pooling2d_1/MaxPool:0' shape=(55000, 7, 7, 64) dtype=float32>

In [15]:
#攤平矩正

pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

In [16]:
pool2_flat

<tf.Tensor 'Reshape_1:0' shape=(55000, 3136) dtype=float32>

In [17]:
# 用 1024個神經元去吸附這些特徵

dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

In [18]:
dense

<tf.Tensor 'dense/Relu:0' shape=(55000, 1024) dtype=float32>

In [19]:
# 學習 0.4 的區域，也就代表會剃除 0.6 的內容

dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=True)

In [20]:
dropout

<tf.Tensor 'dropout/dropout/mul:0' shape=(55000, 1024) dtype=float32>

In [21]:
# 每 10個單位紀錄一筆學習成績

logits = tf.layers.dense(inputs=dropout, units=10)

In [22]:
# 錯誤比照

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)

In [23]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

[tf.train.get_global_step](https://www.tensorflow.org/api_docs/python/tf/train/get_global_step)

In [24]:
train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())

In [25]:
sess = tf.Session()

In [ ]:
init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
sess.run(train_op)

In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.save(sess, "mnist", global_step=0)

## mnist [教學](https://www.tensorflow.org/tutorials/layers)

## code [link](https://github.com/tensorflow/tensorflow/blob/r1.7/tensorflow/examples/tutorials/layers/cnn_mnist.py)